Import library boto3 (library AWS SDK yang kompatibel dengan MinIO)

In [1]:
import boto3
import pandas as pd

Setup Path File Parquet

In [2]:
sourceFilePath = r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah/cleanedData/dapodik/dataYangSudahDiKategorisasi.csv'
minioObjectName = 'JawaBarat/dataDKIJakarta.parquet'
targetFilePath = '../parquetStaging/DKIJakarta.parquet'

In [3]:
df = pd.read_csv(sourceFilePath, sep=';')
print(df)

               Nama_Sekolah        Provinsi                     Kota  \
0        SDN DUREN SAWIT 01  D.K.I. Jakarta  Kota Adm. Jakarta Timur   
1     SDN DUREN SAWIT 05 PG  D.K.I. Jakarta  Kota Adm. Jakarta Timur   
2        SDN DUREN SAWIT 06  D.K.I. Jakarta  Kota Adm. Jakarta Timur   
3     SDN DUREN SAWIT 13 PG  D.K.I. Jakarta  Kota Adm. Jakarta Timur   
4     SDN DUREN SAWIT 15 PG  D.K.I. Jakarta  Kota Adm. Jakarta Timur   
...                     ...             ...                      ...   
1748          SMPI AL JIHAD  D.K.I. Jakarta  Kota Adm. Jakarta Pusat   
1749         SMP PASKALIS 1  D.K.I. Jakarta  Kota Adm. Jakarta Pusat   
1750               SMP YWPM  D.K.I. Jakarta  Kota Adm. Jakarta Pusat   
1751        SMKN 14 JAKARTA  D.K.I. Jakarta  Kota Adm. Jakarta Pusat   
1752        SMKN 31 JAKARTA  D.K.I. Jakarta  Kota Adm. Jakarta Pusat   

                  Kecamatan      NPSN  Status Bentuk_Pendidikan  \
0     Kecamatan Duren Sawit  20108557  Negeri                SD   
1

In [4]:
df.to_parquet(targetFilePath)

Setup Kredensial MinIO

In [5]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'data-dapodik-kemendikdasmen',
    'endpoint': 'http://localhost:9000'
}

Inisialisasi Koneksi (Resource dan Client)

In [6]:
# Menggunakan Resource untuk fungsi level tinggi (seperti upload file)
s3Resource = boto3.resource(
    's3',
    endpoint_url = minioAuth['endpoint'],
    aws_access_key_id = minioAuth['accessKey'],
    aws_secret_access_key = minioAuth['secretKey'],
    verify = False # Wajib: Karena MinIO default di Docker itu http / tidak menggunakan SSL (https)
)

# Menggunakan Client untuk fungsi level rendah (seperti cek/buat bucket)
s3_client = boto3.client(
    's3',
    endpoint_url = minioAuth['endpoint'],
    aws_access_key_id = minioAuth['accessKey'],
    aws_secret_access_key= minioAuth['secretKey'],
    verify = False
)

Fungsi untuk memastikan bucket ada

In [7]:
def checkBucketIfExists(bucketName):
    try:
        
        # Jika sudah ada
        s3_client.head_bucket(Bucket = bucketName)
        print(f"Bucket '{bucketName}' sudah ada.")
        
    except Exception as e:
        
        # Jika bucket tidak ditemukan, buat bucket baru
        if '404' in str(e):
             s3_client.create_bucket(Bucket = bucketName)
             print(f"Bucket '{bucketName}' berhasil dibuat.")
             
        # Jika error bukan 404 (artinya ada kesalahan lain, biasanya salah endpoint api)
        else:
            print(f"Terjadi kesalahan saat memeriksa/membuat bucket: {e}")

checkBucketIfExists(minioAuth['bucket'])

Bucket 'data-dapodik-kemendikdasmen' sudah ada.


In [9]:
# Upload file cuaca
s3Resource.Bucket(minioAuth['bucket']).upload_file(
    targetFilePath,
    minioObjectName
    )

print("Berhasil mengunggah ke MinIO ✅")
print("-------------------------------")
print(f"\nAnda bisa melihat data pangan di GUI MinIO pada path: {minioAuth['bucket']}/{minioObjectName}")

Berhasil mengunggah ke MinIO ✅
-------------------------------

Anda bisa melihat data pangan di GUI MinIO pada path: data-dapodik-kemendikdasmen/JawaBarat/dataDKIJakarta.parquet
